In [1]:
# load data

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from time import time

from data_util import get_data_set, maybe_download_and_extract
from model_util import model

In [3]:
train_x, train_y, train_l = get_data_set(cifar=10)
test_x, test_y, test_l = get_data_set("test", cifar=10)

x, y, output, global_step, y_pred_cls = model()

_IMG_SIZE = 32
_NUM_CHANNELS = 3
_BATCH_SIZE = 128
_CLASS_SIZE = 10
_ITERATION = 1000
_SAVE_PATH = "/tmp/tensorboard/cifar-10/"


loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=1e-4).minimize(loss, global_step=global_step)


correct_prediction = tf.equal(y_pred_cls, tf.argmax(y, dimension=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar("Accuracy/train", accuracy)


merged = tf.summary.merge_all()
saver = tf.train.Saver()
sess = tf.Session()
train_writer = tf.summary.FileWriter(_SAVE_PATH, sess.graph)



INFO:tensorflow:Summary name Fully connected layers/fc1 is illegal; using Fully_connected_layers/fc1 instead.
INFO:tensorflow:Summary name Fully connected layers/fc1 is illegal; using Fully_connected_layers/fc1 instead.
INFO:tensorflow:Summary name Fully connected layers/fc2 is illegal; using Fully_connected_layers/fc2 instead.
INFO:tensorflow:Summary name Fully connected layers/fc2 is illegal; using Fully_connected_layers/fc2 instead.
INFO:tensorflow:Summary name Fully connected layers/output is illegal; using Fully_connected_layers/output instead.


In [4]:
print(train_x.shape)
print(train_y.shape)
print(train_l[:5])

(50000, 3072)
(50000, 10)
['airplane', 'automobile', 'bird', 'cat', 'deer']


In [5]:
def train(num_iterations):
    '''
        Train CNN
    '''
    for i in range(num_iterations):
        randidx = np.random.randint(len(train_x), size=_BATCH_SIZE)
        batch_xs = train_x[randidx]
        batch_ys = train_y[randidx]

        start_time = time()
        i_global, _ = sess.run([global_step, optimizer], feed_dict={x: batch_xs, y: batch_ys})
        duration = time() - start_time

        if (i_global % 10 == 0) or (i == num_iterations - 1):
            _loss, batch_acc = sess.run([loss, accuracy], feed_dict={x: batch_xs, y: batch_ys})
            msg = "Global Step: {0:>6}, accuracy: {1:>6.1%}, loss = {2:.2f} ({3:.1f} examples/sec, {4:.2f} sec/batch)"
            print(msg.format(i_global, batch_acc, _loss, _BATCH_SIZE / duration, duration))

        if (i_global % 100 == 0) or (i == num_iterations - 1):
            
            
            data_merged, global_1 = sess.run([merged, global_step], feed_dict={x: batch_xs, y: batch_ys})
            acc = predict_test()

            summary = tf.Summary(value=[
                tf.Summary.Value(tag="Accuracy/test", simple_value=acc),
            ])
            train_writer.add_summary(data_merged, global_1)
            train_writer.add_summary(summary, global_1)

            saver.save(sess, save_path=_SAVE_PATH, global_step=global_step)
            print("Saved checkpoint.")

In [6]:
def predict_test(show_confusion_matrix=False):
    '''
        Make prediction for all images in test_x
    '''
    i = 0
    predicted_class = np.zeros(shape=len(test_x), dtype=np.int)
    while i < len(test_x):
        j = min(i + _BATCH_SIZE, len(test_x))
        batch_xs = test_x[i:j, :]
        batch_ys = test_y[i:j, :]
        predicted_class[i:j] = sess.run(y_pred_cls, feed_dict={x: batch_xs, y: batch_ys})
        i = j

    correct = (np.argmax(test_y, axis=1) == predicted_class)
    acc = correct.mean()*100
    correct_numbers = correct.sum()
    print("Accuracy on Test-Set: {0:.2f}% ({1} / {2})".format(acc, correct_numbers, len(test_x)))

    if show_confusion_matrix is True:
        cm = confusion_matrix(y_true=np.argmax(test_y, axis=1), y_pred=predicted_class)
        for i in range(_CLASS_SIZE):
            class_name = "({}) {}".format(i, test_l[i])
            print(cm[i, :], class_name)
        class_numbers = [" ({0})".format(i) for i in range(_CLASS_SIZE)]
        print("".join(class_numbers))

    return acc

In [7]:

try:
    print("Trying to restore last checkpoint ...")
    last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=_SAVE_PATH)
    saver.restore(sess, save_path=last_chk_path)
    print("Restored checkpoint from:", last_chk_path)
except:
    print("Failed to restore checkpoint. Initializing variables instead.")
    sess.run(tf.global_variables_initializer())

if _ITERATION != 0:
    train(_ITERATION)

Trying to restore last checkpoint ...
INFO:tensorflow:Restoring parameters from None
Failed to restore checkpoint. Initializing variables instead.
Global Step:     10, accuracy:  12.5%, loss = 2.30 (620.8 examples/sec, 0.21 sec/batch)
Global Step:     20, accuracy:   6.2%, loss = 2.31 (679.5 examples/sec, 0.19 sec/batch)
Global Step:     30, accuracy:  10.9%, loss = 2.30 (885.3 examples/sec, 0.14 sec/batch)
Global Step:     40, accuracy:  12.5%, loss = 2.30 (951.5 examples/sec, 0.13 sec/batch)
Global Step:     50, accuracy:   9.4%, loss = 2.30 (916.9 examples/sec, 0.14 sec/batch)
Global Step:     60, accuracy:   5.5%, loss = 2.30 (885.3 examples/sec, 0.14 sec/batch)
Global Step:     70, accuracy:   9.4%, loss = 2.30 (972.4 examples/sec, 0.13 sec/batch)
Global Step:     80, accuracy:  11.7%, loss = 2.30 (919.2 examples/sec, 0.14 sec/batch)
Global Step:     90, accuracy:  12.5%, loss = 2.30 (884.6 examples/sec, 0.14 sec/batch)
Global Step:    100, accuracy:  12.5%, loss = 2.30 (886.8 exa

Global Step:    870, accuracy:  40.6%, loss = 1.46 (948.0 examples/sec, 0.14 sec/batch)
Global Step:    880, accuracy:  45.3%, loss = 1.58 (927.9 examples/sec, 0.14 sec/batch)
Global Step:    890, accuracy:  42.2%, loss = 1.54 (890.8 examples/sec, 0.14 sec/batch)
Global Step:    900, accuracy:  39.8%, loss = 1.55 (868.8 examples/sec, 0.15 sec/batch)
Accuracy on Test-Set: 40.53% (4053 / 10000)
Saved checkpoint.
Global Step:    910, accuracy:  39.1%, loss = 1.77 (931.7 examples/sec, 0.14 sec/batch)
Global Step:    920, accuracy:  46.1%, loss = 1.48 (972.3 examples/sec, 0.13 sec/batch)
Global Step:    930, accuracy:  35.9%, loss = 1.58 (919.6 examples/sec, 0.14 sec/batch)
Global Step:    940, accuracy:  46.9%, loss = 1.50 (966.5 examples/sec, 0.13 sec/batch)
Global Step:    950, accuracy:  45.3%, loss = 1.45 (928.0 examples/sec, 0.14 sec/batch)
Global Step:    960, accuracy:  44.5%, loss = 1.48 (898.3 examples/sec, 0.14 sec/batch)
Global Step:    970, accuracy:  44.5%, loss = 1.57 (967.8 

Global Step:   1740, accuracy:  56.2%, loss = 1.16 (932.4 examples/sec, 0.14 sec/batch)
Global Step:   1750, accuracy:  44.5%, loss = 1.42 (935.4 examples/sec, 0.14 sec/batch)
Global Step:   1760, accuracy:  50.0%, loss = 1.41 (924.2 examples/sec, 0.14 sec/batch)
Global Step:   1770, accuracy:  50.8%, loss = 1.27 (946.8 examples/sec, 0.14 sec/batch)
Global Step:   1780, accuracy:  49.2%, loss = 1.36 (930.1 examples/sec, 0.14 sec/batch)
Global Step:   1790, accuracy:  63.3%, loss = 1.12 (881.1 examples/sec, 0.15 sec/batch)
Global Step:   1800, accuracy:  50.8%, loss = 1.32 (964.9 examples/sec, 0.13 sec/batch)
Accuracy on Test-Set: 50.24% (5024 / 10000)
Saved checkpoint.
Global Step:   1810, accuracy:  58.6%, loss = 1.17 (926.4 examples/sec, 0.14 sec/batch)
Global Step:   1820, accuracy:  57.8%, loss = 1.21 (931.3 examples/sec, 0.14 sec/batch)
Global Step:   1830, accuracy:  51.6%, loss = 1.33 (1011.3 examples/sec, 0.13 sec/batch)
Global Step:   1840, accuracy:  53.1%, loss = 1.20 (939.7

In [8]:
predict_test(True)

Accuracy on Test-Set: 53.71% (5371 / 10000)
[571  52  46  21   4   7  12   7 222  58] (0) airplane
[ 18 742   2  12   1   2   9   4  63 147] (1) automobile
[ 98  26 354 142  67 110  81  46  46  30] (2) bird
[ 32  25  88 470  15 201  55  22  37  55] (3) cat
[ 56  11 183 117 230  99 135 114  39  16] (4) deer
[ 21  11  89 268  14 469  23  55  25  25] (5) dog
[ 10  21  61 180  25  20 613   5  20  45] (6) frog
[ 31  13  44  86  46 132  19 536  17  76] (7) horse
[ 73  73  13  12   0   6   5   3 769  46] (8) ship
[ 22 217  10  24   1   5  12   9  83 617] (9) truck
 (0) (1) (2) (3) (4) (5) (6) (7) (8) (9)


53.710000000000001

In [24]:
g = tf.get_default_graph()
feed = g.get_tensor_by_name('data/Input:0')
fetch = g.get_tensor_by_name('output/output:0')

# Feeding 3 images through the net just for testing
feed_vals = train_x[0:3]
res = sess.run(fetch, feed_dict={feed:feed_vals})
np.shape(feed_vals), res.shape

((3, 3072), (3, 10))

In [28]:
# for embedding
N = 50000
p = 10
EMB = np.zeros((N, p), dtype='float32')
for i in range(N): #Of course you could do mini-batches
    EMB[i] = sess.run(fetch, feed_dict={feed: train_x[i:i+1,:]})
    if (i % 5000 == 0 or i < 5):
        print(i)

0
1
2
3
4
5000
10000
15000
20000
25000
30000
35000
40000
45000


In [42]:
LOG_DIR = _SAVE_PATH
from tensorflow.contrib.tensorboard.plugins import projector
import os
# The embedding variable, which needs to be stored
# Note this must a Variable not a Tensor!
embedding_var = tf.Variable(EMB,  name='Embedding_of_output')
sess.run(embedding_var.initializer)
summary_writer = tf.summary.FileWriter(LOG_DIR)
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

# Comment out if you don't have metadata
embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')

projector.visualize_embeddings(summary_writer, config)
saver = tf.train.Saver([embedding_var])
saver.save(sess, os.path.join(LOG_DIR, 'model2.ckpt'), 1)

'/tmp/tensorboard/cifar-10/model2.ckpt-1'

In [43]:
metadata_file = open(os.path.join(LOG_DIR, 'metadata.tsv'), 'w')
metadata_file.write('Name\tClass\n')
print(train_y[:5])
print(train_l)

print(train_l[np.argmax(train_y[0])])
print(train_l[np.argmax(train_y[1])])
print(train_l[np.argmax(train_y[2])])

for i in range(N):
    metadata_file.write('%06d\t%s\n' % (i, train_l[np.argmax(train_y[i])]))
metadata_file.close()

[[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]]
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
frog
truck
truck


In [44]:
%ls -lhl /tmp/tensorboard/cifar-10

total 99M
-rw-rw-r-- 1 jac jac  16M Sep 18 18:00 -1600.data-00000-of-00001
-rw-rw-r-- 1 jac jac 1.9K Sep 18 18:00 -1600.index
-rw-rw-r-- 1 jac jac 150K Sep 18 18:00 -1600.meta
-rw-rw-r-- 1 jac jac  16M Sep 18 18:01 -1700.data-00000-of-00001
-rw-rw-r-- 1 jac jac 1.9K Sep 18 18:01 -1700.index
-rw-rw-r-- 1 jac jac 150K Sep 18 18:01 -1700.meta
-rw-rw-r-- 1 jac jac  16M Sep 18 18:01 -1800.data-00000-of-00001
-rw-rw-r-- 1 jac jac 1.9K Sep 18 18:01 -1800.index
-rw-rw-r-- 1 jac jac 150K Sep 18 18:01 -1800.meta
-rw-rw-r-- 1 jac jac  16M Sep 18 18:01 -1900.data-00000-of-00001
-rw-rw-r-- 1 jac jac 1.9K Sep 18 18:01 -1900.index
-rw-rw-r-- 1 jac jac 150K Sep 18 18:01 -1900.meta
-rw-rw-r-- 1 jac jac  16M Sep 18 18:01 -2000.data-00000-of-00001
-rw-rw-r-- 1 jac jac 1.9K Sep 18 18:01 -2000.index
-rw-rw-r-- 1 jac jac 150K Sep 18 18:01 -2000.meta
-rw-rw-r-- 1 jac jac  135 Sep 18 18:28 checkpoint
-rw-rw-r-- 1 jac jac 767K Sep 18 18:01 events.out.tfevents.1505728544.jac-T5-SKYLAKE
-rw-rw-r-- 1 jac jac 635K

In [45]:
sess.close()